In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
from google.colab import drive
from sklearn.preprocessing import StandardScaler
from google.colab import drive
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans
import xgboost as xgb
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# csv locations can be updated
df_html_0 = pd.read_csv("/content/drive/MyDrive/dataminingPhising/features/html_features_0_10000.csv")  # Load your dataset
df_html_10000 = pd.read_csv("/content/drive/MyDrive/dataminingPhising/features/html_features_0_10000.csv")  # Load your dataset
df_html_20000 = pd.read_csv("/content/drive/MyDrive/dataminingPhising/features/html_features_10000_20000.csv")  # Load your dataset
df_html_30000 = pd.read_csv("/content/drive/MyDrive/dataminingPhising/features/html_features_20000_30000.csv")  # Load your dataset

In [3]:
df_features_0 = pd.read_csv("/content/drive/MyDrive/dataminingPhising/features/url_features_0_10000.csv")
df_features_10000 = pd.read_csv("/content/drive/MyDrive/dataminingPhising/features/url_features_10000_20000.csv")
df_features_20000 = pd.read_csv("/content/drive/MyDrive/dataminingPhising/features/url_features_20000_30000.csv")
df_features_30000 = pd.read_csv("/content/drive/MyDrive/dataminingPhising/features/url_features_30000_80000.csv")

In [4]:
# Merge all dataframes into one
merged_df_html = pd.concat([df_html_0 , df_html_10000, df_html_20000, df_html_30000], ignore_index=True)
merged_df_features = pd.concat([df_features_0, df_features_10000, df_features_20000, df_features_30000], ignore_index=True)


In [5]:
def preprocess_html(train_df):
  columns_to_remove = [
    'phishing', 'rec_id', 'ExtFavicon',
    ]

  # Split labels and features
  labels = train_df['phishing']
  features = train_df.drop(columns_to_remove, axis=1)

  # Split the data into training and testing sets
  X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.4, random_state=42)

  columns_to_scale = [
    'total_forms', 'total_hyperlinks'
  ]

  # Standardize the features
  scaler = StandardScaler()
  X_train[columns_to_scale] = scaler.fit_transform(X_train[columns_to_scale])
  X_test[columns_to_scale] = scaler.transform(X_test[columns_to_scale])

  columns_to_clusters_k15 = ['script_files_ratio',	'css_files_ratio'	,'image_files_ratio',	'anchor_files_ratio',	'empty_anchor_ratio',	'null_hyperlink_ratio',
  'internal_hyperlink_ratio'	,'external_hyperlink_ratio'	,'external_to_internal_ratio', 'PctNullSelfRedirectHyperlinks'
                         ]

  columns_to_clusters_k5 = ['suspicious_form_ratio'	,'PctExtHyperlinks'	,'PctExtResourceUrls']

  X_k15 = X_train[columns_to_clusters_k15]
  X_k5 = X_train[columns_to_clusters_k5]

  kmeans_k15 = KMeans(n_clusters=15, random_state=42)
  kmeans_k5 = KMeans(n_clusters=5, random_state=42)

  kmeans_k15.fit(X_k15)
  kmeans_k5.fit(X_k5)


  X_train['cluster_k15'] = kmeans_k15.predict(X_train[columns_to_clusters_k15])  # Use predict for X_train
  X_train['cluster_k5'] = kmeans_k5.predict(X_train[columns_to_clusters_k5])  # Use predict for X_train
  X_test['cluster_k15'] = kmeans_k15.predict(X_test[columns_to_clusters_k15])  # Use predict for X_test
  X_test['cluster_k5'] = kmeans_k5.predict(X_test[columns_to_clusters_k5])  # Use predict for X_test


  return X_train, y_train, X_test, y_test



In [6]:
html_train_X, html_train_y, html_test_X, html_test_y = preprocess_html(merged_df_html)


In [7]:
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "k-NN (k=1)": KNeighborsClassifier(n_neighbors=1),
    "k-NN (k=3)": KNeighborsClassifier(n_neighbors=3),
    "Gaussian Naive Bayes": GaussianNB(),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "MLP Neural Network": MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": xgb.XGBClassifier(random_state=42)
}

In [8]:
def evaluate_models(models, X_train, y_train, X_test, y_test):
  for model_name, model in models.items():
      # Train the model
      model.fit(X_train, y_train)

      # Predict on the training set
      y_train_pred = model.predict(X_train)
      # Calculate training accuracy
      train_accuracy = accuracy_score(y_train, y_train_pred)

      # Predict on the test set
      y_test_pred = model.predict(X_test)
      # Calculate testing accuracy
      test_accuracy = accuracy_score(y_test, y_test_pred)
      print(f"ID of {model_name}: {id(model)}")  # Print the ID

      # Print the accuracies for each model
      print(f"{model_name} - Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")

In [9]:
# evaluate_models(models, features_train_X, features_train_y, features_test_X, features_test_y)
evaluate_models(models, html_train_X, html_train_y, html_test_X, html_test_y)

ID of Decision Tree: 135598674290784
Decision Tree - Train Accuracy: 0.9830, Test Accuracy: 0.9366
ID of k-NN (k=1): 135598674293232
k-NN (k=1) - Train Accuracy: 0.9645, Test Accuracy: 0.9222
ID of k-NN (k=3): 135598674290016
k-NN (k=3) - Train Accuracy: 0.9383, Test Accuracy: 0.9074
ID of Gaussian Naive Bayes: 135598674294336
Gaussian Naive Bayes - Train Accuracy: 0.8307, Test Accuracy: 0.8267
ID of Logistic Regression: 135598674823728
Logistic Regression - Train Accuracy: 0.8698, Test Accuracy: 0.8674
ID of MLP Neural Network: 135598674823104
MLP Neural Network - Train Accuracy: 0.9575, Test Accuracy: 0.9330
ID of Random Forest: 135598674824112
Random Forest - Train Accuracy: 0.9830, Test Accuracy: 0.9527
ID of XGBoost: 135598674822672
XGBoost - Train Accuracy: 0.9768, Test Accuracy: 0.9473


In [10]:
models2 = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "k-NN (k=1)": KNeighborsClassifier(n_neighbors=1),
    "k-NN (k=3)": KNeighborsClassifier(n_neighbors=3),
    "Gaussian Naive Bayes": GaussianNB(),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "MLP Neural Network": MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": xgb.XGBClassifier(random_state=42)
}

In [11]:
def preprocess_features(train_df):
  columns_to_remove = [
      'phishing', 'rec_id',
    'qty_/_domain', 'qty_?_domain', 'qty_=_domain', 'qty_@_domain',
    'qty_&_domain', 'qty_!_domain', 'qty_ _domain', 'qty_~_domain',
    'qty_,_domain', 'qty_+_domain', 'qty_*_domain', 'qty_#_domain',
    'qty_$_domain', 'qty_%_domain',
      'time_response'	,'domain_spf'	,'asn_ip'	,'time_domain_activation'	,
      'time_domain_expiration'	,'qty_ip_resolved'	,'qty_nameservers',
      'qty_mx_servers',	'ttl_hostname'	,'tls_ssl_certificate',
      'qty_redirects',	'url_google_index',	'domain_google_index',	'url_shortened'
]

   # Split labels and features
  labels = train_df['phishing']
  features = train_df.drop(columns_to_remove, axis=1)

  # Split the data into training and testing sets
  X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.4, random_state=42)

  scaler = StandardScaler()
  X_train  = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  return X_train, y_train, X_test, y_test



In [12]:
features_train_X, features_train_y, features_test_X, features_test_y = preprocess_features(merged_df_features)


In [13]:
evaluate_models(models2, features_train_X, features_train_y, features_test_X, features_test_y)

ID of Decision Tree: 135598655978656
Decision Tree - Train Accuracy: 0.9836, Test Accuracy: 0.8818
ID of k-NN (k=1): 135598656194256
k-NN (k=1) - Train Accuracy: 0.9781, Test Accuracy: 0.8902
ID of k-NN (k=3): 135598656194304
k-NN (k=3) - Train Accuracy: 0.9365, Test Accuracy: 0.8947
ID of Gaussian Naive Bayes: 135598656194352
Gaussian Naive Bayes - Train Accuracy: 0.7076, Test Accuracy: 0.7078
ID of Logistic Regression: 135598656197040
Logistic Regression - Train Accuracy: 0.8456, Test Accuracy: 0.8465
ID of MLP Neural Network: 135598656196992
MLP Neural Network - Train Accuracy: 0.9474, Test Accuracy: 0.9098
ID of Random Forest: 135598656196944
Random Forest - Train Accuracy: 0.9836, Test Accuracy: 0.9145
ID of XGBoost: 135598656196848
XGBoost - Train Accuracy: 0.9323, Test Accuracy: 0.9150
